In [1]:
from text2brick.models import GraphLegoWorldData
import numpy as np
import torch
from text2brick.dataset.dataset import MNISTDataset
from text2brick.gym import LegoEnv


In [2]:
table = np.array([[0, 0, 0, 0],
                  [1, 1, 0, 0],
                  [0, 1, 1, 0],
                  [1, 1, 0, 0]])

In [8]:
# dataset = MNISTDataset()
# array, _, _, _ = dataset.sample(sample_index=1)

In [9]:
# env = LegoEnv(array.shape[0])
# print(env)

In [10]:
# obs, reward, done, info = env.step((0, 14), array) # Valid
# obs, reward, done, info = env.step((0, 14), array) # Overlap
# obs, reward, done, info = env.step((0, 27), array) # Half of the brick is out of the world 
# obs, reward, done, info = env.step((3, 14), array) # Invalid
# obs, reward, done, info = env.step((1, 14), array) # Valid
# print(obs)

In [3]:
graph = GraphLegoWorldData(img=table)
graph.add_brick(3, 0)
graph.add_brick(2, 0)
graph.print_graph()
graph.save_as_ldraw()

Outside: Brick at (3, 0) is out of the world
Number of nodes: 4
Number of edges: 3
Node 0: {'x': 0, 'y': 0, 'validity': True}
Node 1: {'x': 1, 'y': 1, 'validity': True}
Node 2: {'x': 0, 'y': 2, 'validity': True}
Node 3: {'x': 2, 'y': 0, 'validity': True}
Edge (0, 1): {}
Edge (1, 2): {}
Edge (1, 3): {}


In [16]:
sub = graph.subgraph(3)
for node, data in sub.nodes(data=True):
    print(f"Node {node}: {data}")
for u, v,  in sub.edges():
    print(f"Edge ({u}, {v}):")

sub

Node 0: {'x': 0, 'y': 0, 'validity': True}
Node 1: {'x': 1, 'y': 1, 'validity': True}
Node 2: {'x': 0, 'y': 2, 'validity': True}
Edge (0, 1):
Edge (1, 2):


In [13]:
graph.graph_to_table()

array([[0, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 1, 0],
       [1, 1, 1, 1]])

In [14]:
data = graph.graph_to_torch()
c = torch.stack([data.x, data.y])
data.edge_index

tensor([[0, 1, 1],
        [1, 2, 3]])

In [1]:
from torch.utils.data import DataLoader
from text2brick.dataset import CustomDatasetGraph

dataset = CustomDatasetGraph()


In [3]:
train_data = DataLoader(dataset, batch_size=1, shuffle=True)

for i in range(2):
    x, edge_index, next_node = next(iter(train_data))
    print(f"Sample {i + 1}:")
    print(f"Node Features (x): {x}")
    print(f"Edge Index: {edge_index}")
    print(f"Next Node: {next_node}")
    print("-" * 30)

Sample 1:
Node Features (x): tensor([[[ 7,  9, 11,  7,  9, 11,  7,  9, 11],
         [ 0,  0,  0,  1,  1,  1,  2,  2,  2]]])
Edge Index: tensor([[[0, 1, 2, 3, 4, 5],
         [3, 4, 5, 6, 7, 8]]])
Next Node: tensor([[-1, -1]], dtype=torch.int32)
------------------------------
Sample 2:
Node Features (x): tensor([[[ 9, 11, 10, 10, 11, 11, 11, 13, 12, 12, 12, 14, 13, 13, 15, 13, 15,
          14, 14, 16, 14],
         [ 0,  0,  1,  2,  3,  4,  5,  5,  6,  7,  8,  8,  9, 10, 10, 11, 11,
          12, 13, 13, 14]]])
Edge Index: tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
          17, 18],
         [ 2,  2,  3,  4,  5,  6,  8,  8,  9, 10, 12, 12, 13, 15, 16, 17, 17,
          18, 20]]])
Next Node: tensor([[16, 14]], dtype=torch.int32)
------------------------------
